# Import Required Libraries

In [ ]:
import mne
import pandas as pd
from glob2 import glob

## Loading normal (non-epilepsy) & abnormal(epilepsy) EDF files from NMT DATASET

In [ ]:
normal_edf_files = glob('Dataset/normal/train/*.edf')
abnormal_edf_files = glob('Dataset/abnormal/train/*.edf')

# Extract normal, abnormal patient IDs and combine them for All patients ID list

In [ ]:
normal_patient_id = list(set([x.split('/')[-1].split('.')[0] for x in normal_edf_files]))
abnormal_patient_id = list(set([x.split('/')[-1].split('.')[0] for x in abnormal_edf_files]))
all_patient_id = abnormal_patient_id + normal_patient_id

# Sampling frequency is 200.0 hz according to NMT dataset. Window length of 10 second is chosen

In [1]:
SAMPLING_FREQ = 200.0
WINDOW_LENGTH_SECONDS = 20.0
WINDOW_LENGTH_SAMPLES = int(WINDOW_LENGTH_SECONDS * SAMPLING_FREQ)

dataset_index_rows = []


# Information from the EDF files are extracted, labaled and saved in a dictionary

In [ ]:
for idx, patient in enumerate(all_patient_id):
    print(f"{patient}: {idx+1}/{len(all_patient_id)}\n\n")
    if patient in normal_patient_id:
        label = 'no_epilepsy'
        print(label)
        raw_file_path = f'Dataset/normal/{patient}.edf'
        print(raw_file_path)
        raw_data = mne.io.read_raw_edf(
            raw_file_path, verbose=False, preload=False)
    elif patient in abnormal_patient_id:
        label = 'epilespy'
        print(label)
        raw_file_path = f'Dataset/abnormal/{patient}.edf'
        print(raw_file_path)
        raw_data = mne.io.read_raw_edf(
            raw_file_path, verbose=False, preload=False)

    for start_sample_index in range(0, int(int(raw_data.times[-1]) * SAMPLING_FREQ), WINDOW_LENGTH_SAMPLES):
        end_sample_index = start_sample_index + (WINDOW_LENGTH_SAMPLES - 1)

        # ensure 10 seconds are available in window and recording does not end
        if end_sample_index > raw_data.n_times:
            break

    row = {}
    row["patient_id"] = patient
    row["raw_file_path"] = raw_file_path
    row["record_length_seconds"] = raw_data.times[-1]
    # this is the desired SFREQ using which sample indices are derived.
    # CAUTION - this is not the original SFREQ at which the data is recorded.
    row["sampling_freq"] = SAMPLING_FREQ
    row["channel_config"] = '02_tcp_le'
    row["start_sample_index"] = start_sample_index
    row["end_sample_index"] = end_sample_index
    row["text_label"] = label
    row["numeric_label"] = 0 if label == "no_epilepsy" else 1
    dataset_index_rows.append(row)


# Patient information is saved as `nmt_data.csv` under data folder

In [ ]:
df = pd.DataFrame(dataset_index_rows, columns=["patient_id",
                                               "raw_file_path",
                                               "record_length_seconds",
                                               "sampling_freq",
                                               "channel_config",
                                               "start_sample_index",
                                               "end_sample_index",
                                               "text_label",
                                               "numeric_label"])
df.to_csv("nmt_data.csv", index=False)